<a href="https://colab.research.google.com/github/armandovl/curso_nlp/blob/main/Curso_NLP_3_1_Asociacion_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://i1.wp.com/www.sopitas.com/wp-content/uploads/2014/08/Tec-de-Monterrey-Logo-640x581.jpg" width="70"/> 
<h5>ITESM Campus Santa Fe <br> Armando Valdés </h5>

### Nomenclatura
🟠 Modificar <br>
⏮ Repetir en caso de cambiar texto<br>
🔘 Opcional

### 1.-Importar las bibliotecas

In [ ]:
import pandas as pd #pandas para Data Frames

In [ ]:
#para trabajar con el texto
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize #para tokenizar en oraciones
from nltk.stem import SnowballStemmer #para derivar

In [ ]:
nltk.download('punkt') #complemeto de nltk

In [ ]:
#para identificar tópicos
import gensim  
from gensim import corpora, models

### 2.-Importar archivo 🟠 

In [ ]:
archivo=open("malos_soluble.txt","r")
texto_del_archivo=archivo.read()

### 3.- Procesar el texto ⏮ 
Convertir texto en frases de data frames

In [ ]:
frases=sent_tokenize(texto_del_archivo) #tokenizar en frases
frases=pd.DataFrame({"Frases":frases}) #convertir en un data Frame
derivar=SnowballStemmer("spanish") #traemos la funcion para derivar

### 4.-Procesar las frases 🟠⏮  <br>
Cambiar: longitud mínima de la palabra 3 <br>
Cambiar : si se quiere aplicar derivación


In [ ]:
#Eliminar stopwords
#Quedarse con las palabras con una longitud superior a 4
#Derivar si se desea

def preproceso(texto_del_archivo):
  resultado=[]
  for token in gensim.utils.simple_preprocess(texto_del_archivo):
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>4: #cambiar la longitud
      #resultado.append(derivar.stem(token)) #cambiar para Derivación Regresiva
      resultado.append(token)
  return resultado

In [ ]:
#⏮ 
frasesprocesadas=frases["Frases"].map(preproceso)
frasesprocesadas

### 5.- Hacer diccionarios 🟠⏮ <br>
cambiar: mínimo de apariciones  no_below=10 <br>
(mínimo 10 veces)<br>
cambiar % que ocupa en una frase no_above=0.5 <br>
(no ocupe más del 50% de la frase)

In [ ]:
diccionario=gensim.corpora.Dictionary(frasesprocesadas)
diccionario.filter_extremes(no_below=5, no_above=0.7,keep_n=100000)
corpus=[diccionario.doc2bow(doc) for doc in frasesprocesadas]

### 6.-Aplicar modelo LDA 🟠⏮ 
Cambiar número de tópicos deseados, y argumentos

In [ ]:
lda_model= gensim.models.LdaMulticore(corpus,num_topics=5,id2word=diccionario,passes=3,workers=3)#semilla ,random_state=420
for idx,topic in lda_model.print_topics(-1):
  print("Tema:{} \nPalabras: {} \n" .format(idx,topic))

In [ ]:
#coherencia mas cercano a 1, mayor coherencia
from gensim.models.coherencemodel import CoherenceModel
cm = CoherenceModel(model=lda_model, corpus=corpus,texts="café",coherence='u_mass')
coherence = cm.get_coherence()  # get coherence value
print(coherence)

### 7.-Aplicar modelo LDA alternativo 🟠⏮ 
Cambiar número de tópicos deseados, y argumentos

El corpus alternatvio se basa el el modelo tfidf <br>
Frecuencia de los términos inversa a la frecuencia de las frases <br>
Darle más peso aquellas palabras más representativas, compensando el efecto de que haya muchas frases muy largas con muchas palabras

In [ ]:
#corpus alternativo
tfidf= models.TfidfModel(corpus)
corpus_tfidf=tfidf[corpus]

In [ ]:
lda_model= gensim.models.LdaMulticore(corpus_tfidf,num_topics=6,id2word=diccionario,passes=3,workers=3)#,random_state=223
for idx,topic in lda_model.print_topics(-1):
  print("Tema:{} \nPalabras: {} \n" .format(idx,topic))